In [ ]:
# Enter the sd2e SynBioHub password
SBH_USER = 'sd2e'
SBH_PASSWORD = 'INSERT_PASSWORD'

In [ ]:
import sys
import logging

logging.root.level = logging.WARNING
# logging.root.level = logging.INFO

In [ ]:
# Upgrade pip because sbh-explorer requires version 18+ to process the synbiohub_adapter dependency
try:
    import pip
    pip_major = int(pip.__version__.split('.')[0])
    # Need pip to be at least version 18 to have PEP 508 dependency management URLs
    if pip_major < 18:
        raise ImportError()
except ImportError:
    # Install the dictionary writer library
    !{sys.executable} -m pip install --user --upgrade pip

In [ ]:
# Install sbh-explorer
!{sys.executable} -m pip install --user --upgrade git+https://github.com/SD2E/sbh-explorer

In [ ]:
import synbiohub_adapter as sbha
import sbh_explorer as sbhe

In [ ]:
# Authenticate to SynBioHub

sbh_query = sbha.SynBioHubQuery(sbha.SD2Constants.SD2_SERVER)
sbh_query.login(SBH_USER, SBH_PASSWORD)

# Upward Traversal
Walk up the graph to find what contains a given URI

In [ ]:
# Find what contains a plasmid
uri = 'https://hub.sd2e.org/user/sd2e/design/YG_plasmid_018/1'
#uri = 'https://hub.sd2e.org/user/sd2e/design/YG_plasmid_002/1'

# Display immediate parent ModuleDefinitions
print('Parents:')
for parent in sorted(sbhe.parent_module_definitions(sbh_query, uri)):
      print('  ', parent)
print('')

# Display ancestors that have no parent Module, the top of the line
print('Roots:')
for root in sorted(sbhe.root_module_definitions(sbh_query, uri)):
    print('  ', root)

In [ ]:
# Find what contains a strain
uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
# uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5992/1'
# uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_7377/1'
# uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_7299/1'

# Display immediate parent ModuleDefinitions
print('Parents:')
for parent in sorted(sbhe.parent_module_definitions(sbh_query, uri)):
      print('  ', parent)
print('')

# Display ancestors that have no parent Module, the top of the line
print('Roots:')
for root in sorted(sbhe.root_module_definitions(sbh_query, uri)):
    print('  ', root)

# Downward traversal
Walk down the graph to see what a URI contains

In [ ]:
# Find what a strain contains

uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
# uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5992/1'
# uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_7377/1'

# Display immediate child modules
print('Child modules of', uri)
for child in sorted(sbhe.child_module_definitions(sbh_query, uri)):
    print('  ', child)

# Display immediate child components
print('')
print('Child components of', uri)
for child in sorted(sbhe.child_component_definitions(sbh_query, uri)):
    print('  ', child)

# Finding contained types

In [ ]:
# Find strains and reagents contained in a given URI

# uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2euw_biofab0x2e17026_group_2/1'
uri = 'https://hub.sd2e.org/user/sd2e/experiment_test/experiment0x2euw_biofab0x2e18527_group_1/1'

# Display contained strains
print('Strains contained in', uri)
for strain in sorted(sbhe.find_contained_strains(sbh_query, uri)):
    print('  ', strain)

print('')

# Display contained reagents
print('Reagents contained in', uri)
for reagent in sorted(sbhe.find_contained_reagents(sbh_query, uri)):
    print('  ', reagent)

In [ ]:
# Customize the search with a user-defined predicate

uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'

# Find all ComponentDefinitions and ModuleDefinitions
print('Everything contained in', uri)
def true_predicate(sbh_query, uri):
    """This predicate returns True for everything"""
    return True

for item in sorted(sbhe.find_contained_items(sbh_query, uri, true_predicate)):
    print('  ', item)

# Use a custom predicate to find only some contained items
print('')
print('Custom search in', uri)
def is_my_plasmid(sbh_query, uri):
    """This predicate looks for YG_plasmid_01*"""
    return uri.startswith('https://hub.sd2e.org/user/sd2e/design/YG_plasmid_01')

for item in sorted(sbhe.find_contained_items(sbh_query, uri, is_my_plasmid)):
    print('  ', item)

# Putting it together

In [ ]:
# Find strains used with a plasmid

uri = 'https://hub.sd2e.org/user/sd2e/design/YG_plasmid_018/1'
print('Finding root module definitions, this might take a while...')
root_mds = sbhe.root_module_definitions(sbh_query, uri)
print('Found %d root module definitions' % (len(root_mds)))

# Search each root to find strains used with this URI
all_strains = set()
for root in sorted(root_mds):
    print('Strains in', root)
    for strain in sorted(sbhe.find_contained_strains(sbh_query, root)):
        print('  ', strain)
        all_strains.add(strain)
    print('')

print('--------------------------------------------------')
print(uri, 'is used with strains:')
for strain in sorted(all_strains):
    print('  ', strain)

In [ ]:
# Find reagents used with the given strain

uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'
roots = sbhe.root_module_definitions(sbh_query, uri)
print('Found %d root module definitions' % (len(roots)))
reagents = []
all_reagents = set()
for root in roots:
    print('Reagents in', root)
    for reagent in sorted(sbhe.find_contained_reagents(sbh_query, root)):
        print('  ', reagent)
        all_reagents.add(reagent)
    print('')

print('--------------------------------------------------')
print(uri, 'is used with reagents:')
for reagent in sorted(all_reagents):
    print('  ', reagent)

# Getting Info about a URI

In [ ]:
uri = 'https://hub.sd2e.org/user/sd2e/design/UWBF_5783/1'

import pandas as pd
pd.DataFrame(sorted(sbhe.subject_info(sbh_query, uri)))

# Debugging info

In [ ]:
print('Cache Info')
print('----------')
print('parent_module_definitions:', sbhe.parent_module_definitions.cache_info())
print('root_module_definitions:', sbhe.root_module_definitions.cache_info())
print('child_module_definitions:', sbhe.child_module_definitions.cache_info())
print('child_component_definitions:', sbhe.child_component_definitions.cache_info())
print('module_is_strain:', sbhe.module_is_strain.cache_info())
print('is_reagent:', sbhe.is_reagent.cache_info())
print('find_contained_reagents:', sbhe.find_contained_reagents.cache_info())
print('find_contained_strains:', sbhe.find_contained_strains.cache_info())